# Setting Up

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
! pip install -q transformers[sentencepiece] fastai ohmeow-blurr nbdev

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 956.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [3]:
! pip install -q onnxruntime onnxruntime-gpu onnxruntime_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.8/200.8 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.0 MB/s eta 0:00:00


In [4]:
import torch
from transformers import AutoModelForSequenceClassification, AutoConfig
from fastai.text.all import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *

/usr/local/lib/python3.10/dist-packages/blurr/text/modeling/question_answering.py:31: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = load_metric("squad")


The repository for squad contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/squad.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [5]:
from tqdm.notebook import tqdm
import numpy as np

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd /content/drive/MyDrive/StackExchange-Scifi-Tags-Classifier/notebooks/bert-base-uncased/

/content/drive/MyDrive/StackExchange-Scifi-Tags-Classifier/notebooks/bert-base-uncased


# Data

In [8]:
df = pd.read_csv("../../data/question_details.csv")
df.head()

,title,url,description,tags
0,Would the One Ring even work for anyone but Sauron?,https://scifi.stackexchange.com/questions/48180/would-the-one-ring-even-work-for-anyone-but-sauron,"The temptation of the One Ring, we quickly learn, is always tied up with power: each character's fantasies of possessing it seem to be about what they would do with great power. Even Sam, as I recall, imagines the power to create great huge sweeping gardens. Boromir wants the power to save Gondor, etc.\nBut does the One Ring actually convey power to anyone but Sauron? It actually seems to diminish its bearers: Bilbo feels ""thin"" and ""stretched"", Sméagol becomes the wretched Gollum, Frodo is never quite the same even after it is destroyed. None of them seem more ""powerful,"" even in the abst...","['the-lord-of-the-rings', 'tolkiens-legendarium', 'the-one-ring']"
1,Could I fake death to stop being Force-choked?,https://scifi.stackexchange.com/questions/102822/could-i-fake-death-to-stop-being-force-choked,"I work for a large government organization. My boss is a total jerk who abuses those working under him. I've tried contacting HR before but they won't do anything about him.\nAnyways, I screwed up again. My jerk boss is going to be super pissed.\nI came out of hyperspace too close to some stupid planet and the rebels were able to detect us. It's not a big deal really but he'll make a big thing about it like he always does when something isn't his fault. If it were his f- up, he would say something like 'I meant to do that' or 'It went exactly as I had foreseen it' — yeah right.\nI'm going ...","['star-wars', 'the-force', 'the-empire-strikes-back']"
2,Who or what was Tom Bombadil?,https://scifi.stackexchange.com/questions/1586/who-or-what-was-tom-bombadil,"The most iconic of Middle-earth questions:\nWho or what was Tom Bombadil?\nWas he an elf? Was he akin to Gandalf, and one of the wizardly stock? Was he something… else?\n","['the-lord-of-the-rings', 'tolkiens-legendarium', 'tom-bombadil']"
3,Why don&#39;t Muggle-born wizards use Muggle technology to fight Death Eaters?,https://scifi.stackexchange.com/questions/2611/why-dont-muggle-born-wizards-use-muggle-technology-to-fight-death-eaters,"We know that wizards in the Harry Potter universe are subject to regular physical harm (e.g. witness injuries from being hit by Bludgers or just colliding with things when playing Quidditch).\nTherefore, it's almost a certainty that an average wizard would be vulnerable to, say, a bunch of bullets. \nThere may likely be spell(s) to protect from projectiles - witness Dumbledore's shield deployed when Voldemort sent a bunch of glass at him during the Duel in the Ministry Atrium at the end of Harry Potter and the Order of the Phoenix (according to Wikia, possibly an unusually powerful Protego...","['harry-potter', 'technology']"
4,Story where the number 3 is the monster?,https://scifi.stackexchange.com/questions/2937/story-where-the-number-3-is-the-monster,"I remember as a child (in the nineties) reading a story where the narrator was a little boy and how he was scared only twice a day – when the clock read 3:33. The 3's would be powerful enough (there were three of them at 3:33) to leave the clock in the boy's room for one minute and do what they want. They terrified the boy for that one minute they were free. \nI don't think it was R.L. Stine, but I cannot remember the author.\nUpdate: It definitely wasn't R.L Stine, but the story was in a similar style.\nI know it was only a 15-20 page story, so maybe it was in a short story collection?\n","['story-identification', 'short-stories']"


In [9]:
df = df.dropna().reset_index(drop=True)
df.shape

(30000, 4)

In [10]:
tags_list = df.tags.to_list()
tag_count = {}
for tags in tags_list:
  tag_list = eval(tags)
  for tag in tag_list:
    if tag in tag_count.keys():
      tag_count[tag] += 1
    else:
      tag_count[tag] = 1
print(f"Number of Tags: {len(tag_count)}")
print(tag_count)

threshold = int(len(df) * 0.002)
rare_tags = [key for key, value in tag_count.items() if value < threshold]
len(rare_tags), rare_tags[:5]

Number of Tags: 2095
{'the-lord-of-the-rings': 1390, 'tolkiens-legendarium': 1904, 'the-one-ring': 113, 'star-wars': 3189, 'the-force': 158, 'the-empire-strikes-back': 117, 'tom-bombadil': 18, 'harry-potter': 3454, 'technology': 170, 'story-identification': 8613, 'short-stories': 2763, 'time-travel': 566, 'groundhog-day': 5, 'suggested-order': 137, 'millennium-falcon': 45, 'back-to-the-future': 104, 'back-to-the-future-ii': 22, 'prediction': 11, 'stormtroopers': 28, 'behind-the-scenes': 210, 'the-chronicles-of-narnia': 54, 'cs-lewis': 27, 'xkcd': 9, 'j-r-r-tolkien': 78, 'dc': 900, 'character-identification': 251, 'justice-league': 21, 'merchandise': 5, 'hogwarts': 224, 'et-the-extra-terrestrial': 6, 'shared-universe': 5, 'yoda': 48, 'a-new-hope': 240, 'douglas-adams': 21, 'the-hitchhikers-guide-to-the-galaxy': 67, 'physics': 48, 'batman': 297, 'the-matrix': 264, 'gandalf': 118, 'isaac-asimov': 127, 'foundation': 36, 'horcrux': 130, 'magical-theory': 100, 'star-trek': 2698, 'star-trek-t

(1935,
 ['tom-bombadil',
  'groundhog-day',
  'millennium-falcon',
  'back-to-the-future-ii',
  'prediction'])

In [11]:
tags_list = df.tags.to_list()
revised_tag_list = []
indices_to_drop = []

for idx, tags in enumerate(tags_list):
  tag_list = eval(tags)
  revised_tags = []

  for tag in tag_list:
    if tag not in rare_tags:
      revised_tags.append(tag)

  if len(revised_tags) == 0:
    indices_to_drop.append(idx)
  else:
    revised_tag_list.append(revised_tags)

df = df.drop(indices_to_drop).reset_index(drop=True)
df['revised_tags'] = revised_tag_list
df.shape

(27493, 5)

In [12]:
revised_tags_list = df.revised_tags.to_list()
revised_tag_count = {}
for tags in revised_tags_list:
  tag_list = tags
  for tag in tag_list:
    if tag in revised_tag_count.keys():
      revised_tag_count[tag] += 1
    else:
      revised_tag_count[tag] = 1
print(f"Number of Tags: {len(revised_tag_count)}")
print(revised_tag_count)

Number of Tags: 160
{'the-lord-of-the-rings': 1390, 'tolkiens-legendarium': 1904, 'the-one-ring': 113, 'star-wars': 3189, 'the-force': 158, 'the-empire-strikes-back': 117, 'harry-potter': 3454, 'technology': 170, 'story-identification': 8613, 'short-stories': 2763, 'time-travel': 566, 'suggested-order': 137, 'back-to-the-future': 104, 'behind-the-scenes': 210, 'j-r-r-tolkien': 78, 'dc': 900, 'character-identification': 251, 'hogwarts': 224, 'a-new-hope': 240, 'the-hitchhikers-guide-to-the-galaxy': 67, 'batman': 297, 'the-matrix': 264, 'gandalf': 118, 'isaac-asimov': 127, 'horcrux': 130, 'magical-theory': 100, 'star-trek': 2698, 'star-trek-tng': 868, 'the-force-awakens': 334, 'uss-enterprise': 61, 'star-wars-legends': 343, 'spaceship': 288, 'marvel': 1884, 'marvel-cinematic-universe': 1117, 'sauron': 111, 'religion': 85, 'comics': 498, 'sith': 136, 'darth-vader': 169, 'return-of-the-jedi': 128, 'revenge-of-the-sith': 124, 'albus-dumbledore': 148, 'severus-snape': 109, 'luke-skywalker': 

In [13]:
encode_tag_types = { key: idx for idx, (key, value) in enumerate(revised_tag_count.items())}
with open("../../deployment/tag_types_encoded.json", "w") as fp:
  json.dump(encode_tag_types, fp)

# We need this because for multilabel classification all tags have possibility to be present in the predictions
categorical_tag_list = []
revised_tags_list = df.revised_tags.to_list()

for revised_tags in revised_tags_list:
  categorical_list = [0] * len(encode_tag_types)
  for tag in revised_tags:
    tag_type_index = encode_tag_types[tag]
    categorical_list[tag_type_index] = 1
  categorical_tag_list.append(categorical_list)

df['tag_cat_list'] = categorical_tag_list
df.head()

,title,url,description,tags,revised_tags,tag_cat_list
0,Would the One Ring even work for anyone but Sauron?,https://scifi.stackexchange.com/questions/48180/would-the-one-ring-even-work-for-anyone-but-sauron,"The temptation of the One Ring, we quickly learn, is always tied up with power: each character's fantasies of possessing it seem to be about what they would do with great power. Even Sam, as I recall, imagines the power to create great huge sweeping gardens. Boromir wants the power to save Gondor, etc.\nBut does the One Ring actually convey power to anyone but Sauron? It actually seems to diminish its bearers: Bilbo feels ""thin"" and ""stretched"", Sméagol becomes the wretched Gollum, Frodo is never quite the same even after it is destroyed. None of them seem more ""powerful,"" even in the abst...","['the-lord-of-the-rings', 'tolkiens-legendarium', 'the-one-ring']","[the-lord-of-the-rings, tolkiens-legendarium, the-one-ring]","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
1,Could I fake death to stop being Force-choked?,https://scifi.stackexchange.com/questions/102822/could-i-fake-death-to-stop-being-force-choked,"I work for a large government organization. My boss is a total jerk who abuses those working under him. I've tried contacting HR before but they won't do anything about him.\nAnyways, I screwed up again. My jerk boss is going to be super pissed.\nI came out of hyperspace too close to some stupid planet and the rebels were able to detect us. It's not a big deal really but he'll make a big thing about it like he always does when something isn't his fault. If it were his f- up, he would say something like 'I meant to do that' or 'It went exactly as I had foreseen it' — yeah right.\nI'm going ...","['star-wars', 'the-force', 'the-empire-strikes-back']","[star-wars, the-force, the-empire-strikes-back]","[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
2,Who or what was Tom Bombadil?,https://scifi.stackexchange.com/questions/1586/who-or-what-was-tom-bombadil,"The most iconic of Middle-earth questions:\nWho or what was Tom Bombadil?\nWas he an elf? Was he akin to Gandalf, and one of the wizardly stock? Was he something… else?\n","['the-lord-of-the-rings', 'tolkiens-legendarium', 'tom-bombadil']","[the-lord-of-the-rings, tolkiens-legendarium]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
3,Why don&#39;t Muggle-born wizards use Muggle technology to fight Death Eaters?,https://scifi.stackexchange.com/questions/2611/why-dont-muggle-born-wizards-use-muggle-technology-to-fight-death-eaters,"We know that wizards in the Harry Potter universe are subject to regular physical harm (e.g. witness injuries from being hit by Bludgers or just colliding with things when playing Quidditch).\nTherefore, it's almost a certainty that an average wizard would be vulnerable to, say, a bunch of bullets. \nThere may likely be spell(s) to protect from projectiles - witness Dumbledore's shield deployed when Voldemort sent a bunch of glass at him during the Duel in the Ministry Atrium at the end of Harry Potter and the Order of the Phoenix (according to Wikia, possibly an unusually powerful Protego...","['harry-potter', 'technology']","[harry-potter, technology]","[0, 0, 0, 0, 0, 

In [14]:
labels = list(encode_tag_types.keys())
len(labels), labels[:5]

(160,
 ['the-lord-of-the-rings',
  'tolkiens-legendarium',
  'the-one-ring',
  'star-wars',
  'the-force'])

# Data Split

In [15]:
splitter = RandomSplitter(valid_pct=0.1, seed=42)
train_ids, valid_ids = splitter(df)
len(train_ids), len(valid_ids)

(24744, 2749)

In [16]:
valid_df = df.loc[valid_ids]
valid_df.head()

,title,url,description,tags,revised_tags,tag_cat_list
22112,Why is Slaanesh so young compared to the other Chaos Gods?,https://scifi.stackexchange.com/questions/16745/why-is-slaanesh-so-young-compared-to-the-other-chaos-gods,"Considering the Chaos Gods are all fueled by negative emotion, and Slaanesh is fueled by one of the most base of all emotions, Lust, I would have expected Slaanesh to be the oldest of the lot.\nHowever, people keep telling me Slaanesh is the youngest, born somewhere past the current RL date while the other Chaos Gods were born in the middle ages of our history.\nCan anyone explain to me how this (to me) unexpected situation came to be?\n",['warhammer-40k'],[warhammer-40k],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
21588,"Book about humanity and four other alien races at war, learn they are seed colonies of dead civilizations",https://scifi.stackexchange.com/questions/192465/book-about-humanity-and-four-other-alien-races-at-war-learn-they-are-seed-colon,"I'm looking for a book I read 6 or 7 years ago - maybe less.\nHumanity is at war with one or more of four alien races, there is a sixth race which is a cross breed of the others. Humans come across one of the races and help them get into space or outer space. Can't remember much else apart from they find a message from their own civilizations many millennia in the past and realize that they were all transplanted to a new galaxy by their original civilizations at an almost non existent technological level due to a galactic war that devastated their original planets.\n","['story-identification', 'books', 'military-sf', 'space-opera']","[story-identification, books]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
21157,Where did the inquisitors get their lightsabers?,https://scifi.stackexchange.com/questions/179481/where-did-the-inquisitors-get-their-lightsabers,"In Star Wars rebels and the new Darth Vader comic we see a lot of the Emperors inquisitors and all of them have their own lightsabers (I think they are all double bladed?), but where did they get them? Did they also have to kill a jedi for the crystal and corrupt it or since they are not real sith did the Emperor provide them with a finished lightsaber or even just the crystal? \n","['star-wars', 'comics', 'star-wars-rebels']","[star-wars, comics]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
10507,Robert Heinlein story that begins with a card game followed by an alarm and a shelter,https://scifi.stackexchange.com/questions/8200/robert-heinlein-story-that-begins-with-a-card-game-followed-by-an-alarm-and-a-sh,"The story begins with a group of people playing a card game of some sort, I can't recall what it was exactly, but it had some sort of team aspect to it. There were three or four players. One of them was a woman and she was nervous about the way she was playing and didn't want to let her partner in the game down. Then some event occurred, probably the alarm went off, and they were forced to stop the game in order to go to the shelter.\nThat's basically all I remember. I started this book when I was at my friend's house and didn't get to finish it or advance any further. I'm 99% sure that th...","['story-identification', 'r

We will be using `valid_df` for all inference testing

# Fastai & Blurr Inference

In [17]:
model_name = "bert-base-uncased"

In [18]:
model_path = f"../../models/{model_name}/question-classifier-stage-1.pkl"
learner_inf = load_learner(model_path)

In [19]:
learner_inf.blurr_predict("Why did Batman do so much better in his second fight with Bane?")

[{'labels': ['dc', 'batman'],
  'scores': [0.9526715278625488, 0.75789874792099],
  'class_indices': [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
 

In [20]:
learner_inf.blurr_predict("Why did Batman do so much better in his second fight with Bane?")[0]['labels']

['dc', 'batman']

## Evaluation

In [21]:
from sklearn import metrics

def metric_measures(test_df, preds):

  targets = [np.asarray(target) for target in test_df['tag_cat_list'].to_list()]
  outputs = [np.asarray(pred) for pred in preds]


  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

  return

In [22]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df)):
  desc = row['description']
  labels = learner_inf.blurr_predict(desc)[0]['labels']
  pred_tags = [0] * len(encode_tag_types)
  for label in labels:
    pred_tags[encode_tag_types[label]] = 1
  preds.append(pred_tags)

preds[0][:20]

  0%|          | 0/2749 [00:00<?, ?it/s]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [23]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.715056179775281
F1 Score (Macro) = 0.14664259281673056


# Convert to ONNX

### ONNX
ONNX (Open Neural Network Exchange) is a open standard for representing machine learning models.
- It allows developers to move models between different frameworks (such as PyTorch, TensorFlow, and Caffe2) without losing performance or accuracy.
ONNX makes it easier to build and run AI models on a variety of hardware, including GPUs, CPUs, and custom accelerators.
- This standard helps eliminate the need for multiple conversion tools and provides a unified representation of the model across different tools and frameworks.
- ONNX is being developed by a collaboration of companies including Microsoft, Facebook, Amazon, and IBM, among others, making it a well-supported and widely-adopted standard.
- Converting to ONNX runtime often makes model faster.

In [24]:
model_path = f"../../models/{model_name}/question-classifier-stage-1.pkl"
learner_inf = load_learner(model_path)

In [25]:
classifier = learner_inf.model.hf_model.eval()

torch.onnx.export(
    classifier,
    torch.LongTensor([[0] * 512]),
    f'../../models/{model_name}/question-classifier.onnx',
    input_names=['input_ids'],
    output_names=['output'],
    opset_version=13,
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence_len'},
        'output': {0: 'batch_size'}
    }
)

In [26]:
from onnxruntime.quantization import quantize_dynamic, QuantType

onnx_model_path = f'../../models/{model_name}/question-classifier.onnx'
quantized_onnx_model_path = f'../../models/{model_name}/question-classifier-quantized.onnx'

quantize_dynamic(
    onnx_model_path,
    quantized_onnx_model_path,
    weight_type=QuantType.QUInt8,
)

# ONNX Inference

## Normal ONNX

In [27]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained(model_name)

class_labels = list(encode_tag_types.keys())

inf_session = rt.InferenceSession(f'../../models/{model_name}/question-classifier.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [28]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['description']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_tags = [0] * len(encode_tag_types)
  for label in labels:
    pred_tags[encode_tag_types[label]] = 1
  preds.append(pred_tags)

  0%|          | 0/2749 [00:00<?, ?it/s]

In [29]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.7146870434880325
F1 Score (Macro) = 0.1464390030692479


## Quantized ONNX

In [30]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained(model_name)

class_labels = list(encode_tag_types.keys())

inf_session = rt.InferenceSession(f'../../models/{model_name}/question-classifier-quantized.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [31]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['description']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_tags = [0] * len(encode_tag_types)
  for label in labels:
    pred_tags[encode_tag_types[label]] = 1
  preds.append(pred_tags)

  0%|          | 0/2749 [00:00<?, ?it/s]

In [32]:
metric_measures(valid_df, preds) #

F1 Score (Micro) = 0.716959982064791
F1 Score (Macro) = 0.15025613232540339
